In [361]:
%matplotlib inline
import numpy as np
from pulp import *
import pandas as pd
import pprint
import csv
MEPS=1.03-6

In [362]:
df=pd.read_csv('shop.csv',header=0,usecols=[0])
df

,店舗
0,Shop0
1,Shop1
2,Shop2
3,Shop3
4,Shop4
5,Shop5


In [363]:
DMUs=df.values
print(DMUs)

[['Shop0']
 ['Shop1']
 ['Shop2']
 ['Shop3']
 ['Shop4']
 ['Shop5']]


In [364]:
dfx=pd.read_csv('shop.csv',header=0,index_col=0,usecols=[0,1,2])
print(dfx)
print('')
datax=dfx.values
print(datax)

       店員数  稼働時間
店舗              
Shop0    5    24
Shop1   10    12
Shop2   20    12
Shop3   20    24
Shop4   30    12
Shop5   50    12

[[ 5 24]
 [10 12]
 [20 12]
 [20 24]
 [30 12]
 [50 12]]


In [365]:
x=dfx

In [366]:
print(dfx)
print('')
datay=dfy.values
print(datay)

       店員数  稼働時間
店舗              
Shop0    5    24
Shop1   10    12
Shop2   20    12
Shop3   20    24
Shop4   30    12
Shop5   50    12

[[ 2]
 [ 6]
 [10]
 [12]
 [12]
 [20]]


In [367]:
x.shape

(6, 2)

In [368]:
dfy=pd.read_csv('shop.csv',header=0,usecols=[3])
dfy

,売り上げ
0,2
1,6
2,10
3,12
4,12
5,20


In [369]:
y=dfy

In [370]:
y.shape

(6, 1)

In [371]:
n,m=x.shape
n,s=y.shape

In [372]:
res=[]
vs_=[]
us_=[]

In [373]:
for o in range(n):
    prob=LpProblem('DMU_'+str(o),LpMaximize)
    #　重み
    v=[LpVariable('v'+str(i),lowBound=0,
                  cat='Continuous') for i in range(m)]
    u=[LpVariable('u'+str(i),lowBound=0,
                  cat='Continuous') for i in range(s)]
    #　目的関数
    prob += lpDot(u,y.iloc[o,:])

    # 制約条件
    prob += lpDot(v,x.iloc[o,:])==1,'Normalize'+str(o)

    for j in range(n):
        prob += lpDot(u,y.iloc[j,:]) <= lpDot(v,x.iloc[j,:])

    prob.solve()
    vs=np.array([v[i].varValue for i in range(m)]) #v*
    vs_.append([vs])
    us=np.array([u[i].varValue for i in range(s)]) #u*
    us_.append([us])
    print('DMU_'+str(o),': ',DMUs[o])
    print('vs_'+str(o),': ',vs)
    print('us_'+str(o),': ',us)
    print(' ')
#     # 参照集合作成
#     (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)

#     res.append([DMUs[o],value(prob.objective),
#                 set(eo),tuple(vs),tuple(us)])

    res.append([DMUs[o],value(prob.objective)])

DMU_0 :  ['Shop0']
vs_0 :  [0.2 0. ]
us_0 :  [0.33333333]
 
DMU_1 :  ['Shop1']
vs_1 :  [0.1 0. ]
us_1 :  [0.16666667]
 
DMU_2 :  ['Shop2']
vs_2 :  [0.04       0.01666667]
us_2 :  [0.1]
 
DMU_3 :  ['Shop3']
vs_3 :  [0.05 0.  ]
us_3 :  [0.08333333]
 
DMU_4 :  ['Shop4']
vs_4 :  [0.025      0.02083333]
us_4 :  [0.075]
 
DMU_5 :  ['Shop5']
vs_5 :  [0.         0.08333333]
us_5 :  [0.05]
 


In [374]:
print(vs_[0])

[array([0.2, 0. ])]


In [375]:
test=np.array(res)
print(test)

[[array(['Shop0'], dtype=object) 0.66666666]
 [array(['Shop1'], dtype=object) 1.0000000199999999]
 [array(['Shop2'], dtype=object) 1.0]
 [array(['Shop3'], dtype=object) 0.9999999959999999]
 [array(['Shop4'], dtype=object) 0.8999999999999999]
 [array(['Shop5'], dtype=object) 1.0]]


In [376]:
df_f=pd.DataFrame(res,columns=['店舗番号','効率値'])
df_s=df_f.sort_values('効率値',ascending=False)
df_s

,店舗番号,効率値
1,[Shop1],1.000000
2,[Shop2],1.000000
5,[Shop5],1.000000
3,[Shop3],1.000000
4,[Shop4],0.900000
0,[Shop0],0.666667


In [462]:
k=0

In [463]:
print(datax[k])
print(datay[k][0])

[ 5 24]
2


In [464]:
print(vs_[k][0])
print(us_[k][0][0])

[0.2 0. ]
0.33333333


In [465]:
x1=LpVariable('x1', lowBound=0.0)
x2=LpVariable('x2', lowBound=0.0)
x3=LpVariable('x3', lowBound=0.0)
x4=LpVariable('x4', lowBound=0.0)
x5=LpVariable('x5', lowBound=0.0)
x6=LpVariable('x6', lowBound=0.0)
prob=LpProblem(name='LP-Sample',sense=LpMinimize)

In [469]:
# 目的関数
prob += x1-x2+x3-x4+x5-x6
# 制約式
T1=vs_[k][0][0]*(datax[k][0]+x1-x2)
T2=vs_[k][0][1]*(datax[k][1]+x3-x4)
T3=us_[k][0][0]*(datay[k][0]+x5-x6)
T4=T1+T2
prob += T3==T4
# prob += (vs_[4][0][0]*(datax[4][0]+x1-x2))+vs_[4][0][1]*(datax[4][1]+x3-x4) = us_[4][0]*(datay[4][0]+x5-x6)

In [470]:
print(prob)
prob.solve()

LP-Sample:
MINIMIZE
1*x1 + -1*x2 + 1*x3 + -1*x4 + 1*x5 + -1*x6 + 0
SUBJECT TO
_C1: - 0.2 x1 + 0.2 x2 + 0.33333333 x5 - 0.33333333 x6 = 0.33333334

_C2: - 0.2 x1 + 0.2 x2 + 0.33333333 x5 - 0.33333333 x6 = 0.33333334

_C3: - 0.2 x1 + 0.2 x2 + 0.33333333 x5 - 0.33333333 x6 = 0.33333334

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous
x5 Continuous
x6 Continuous



-2

In [471]:
print(LpStatus[prob.status])
print('Optimal value=', value(prob.objective))

Unbounded
Optimal value= 0.0


In [472]:
for v in prob.variables():
    print(v.name,'=',value(v))

x1 = 0.0
x2 = 0.0
x3 = 0.0
x4 = 0.0
x5 = 0.0
x6 = 0.0


In [473]:
A=value(x1)-value(x2)
print('x1-x2=',A)
B=value(x3)-value(x4)
print('x3-x4=',B)
C=value(x5)-value(x6)
print('x5-x6=',C)

x1-x2= 0.0
x3-x4= 0.0
x5-x6= 0.0


In [474]:
vs_[k][0][0]*(datax[k][0]+A)+vs_[k][0][1]*(datax[k][1]+B)

1.0

In [475]:
us_[k][0][0]*(datay[k][0]+C)

0.66666666